In [ ]:
import pandas as pd

Original file was in strict open xml spreadsheet file type. Do change it to valid excel workbook or csv to have it work with pandas

In [ ]:
df = pd.read_excel('tax_rate_data.xlsx', sheet_name='tax_rate')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   country_code     88 non-null     object 
 1   tax_type         88 non-null     object 
 2   tax_rate         87 non-null     float64
 3   effective_month  88 non-null     object 
dtypes: float64(1), object(3)
memory usage: 2.9+ KB


For the interview, I was tasked to output data where country_code is 'ID' and to do an analysis of the dataset
* What's wrong with the data shown?

In [4]:
indonesia = df[df['country_code'] =='ID']
temp = indonesia.copy()

There are duplicated entries

In [9]:
indonesia[indonesia.duplicated() == True]

,country_code,tax_type,tax_rate,effective_month
24,ID,sbs_outright_revenue,0.1,beginning


Indonesia has incomplete tax_types
* I am also assuming that all tax types here are the different types of fees
* I am also assuming that all tax types are present in the dataset

In [5]:
total_num_taxtype = df['tax_type'].nunique()
total_ID_taxtype = temp['tax_type'].nunique()
print("Total number of tax type: ", total_num_taxtype)
print("Total number of tax type in Indonesia: ", total_ID_taxtype)

Total number of tax type:  17
Total number of tax type in Indonesia:  14


For the tax_rate column
1. Having negative and overly high values does not make logical sense at all (-1% and 130%)
2. Presence of null values
3. Fluctuating tax_rate for cb_handling_fee_revenue, which is odd

In [6]:
temp['tax_rate'].unique()

array([ 0.1  ,  0.11 ,  0.09 ,  0.15 ,  0.18 , -0.01 ,  0.011,  0.01 ,
        1.3  ,    nan])

In [7]:
temp[temp['tax_type'] == 'cb_handling_fee_revenue']

,country_code,tax_type,tax_rate,effective_month
2,ID,cb_handling_fee_revenue,0.10,beginning
3,ID,cb_handling_fee_revenue,0.09,2022-03-01 00:00:00
4,ID,cb_handling_fee_revenue,0.15,2022-04-01 00:00:00
5,ID,cb_handling_fee_revenue,0.11,2022-05-01 00:00:00
6,ID,cb_handling_fee_revenue,0.18,2022-06-01 00:00:00


effective_month has mutiple issues [Visual Inspection]
1. It is incomplete (Suppose to have every month of the year 2022 but only has some)
2. Has mixed data (has data from other years, not just 2022)
    * For data that is labelled as 'beginning', how are we sure that it is infact the beginning of 2022 and not some other year
3. Does not have consistent data format
    * Suppose to be in MMM-YY format but some data is set as 'beginning'

In [8]:
temp

,country_code,tax_type,tax_rate,effective_month
0,ID,cb_comm_revenue,0.100,beginning
1,ID,cb_comm_revenue,0.110,2022-04-01 00:00:00
2,ID,cb_handling_fee_revenue,0.100,beginning
3,ID,cb_handling_fee_revenue,0.090,2022-03-01 00:00:00
4,ID,cb_handling_fee_revenue,0.150,2022-04-01 00:00:00
5,ID,cb_handling_fee_revenue,0.110,2022-05-01 00:00:00
6,ID,cb_handling_fee_revenue,0.180,2022-06-01 00:00:00
7,ID,cb_paid_ads_revenue,0.100,beginning
8,ID,cb_paid_ads_revenue,0.110,2022-04-01 00:00:00
9,ID,digital_product_revenue,0.100,beginning


In [ ]:
# An attempt at cleaning the effective_month column to ensure consistent data format
mask = temp['effective_month'] == 'beginning'
temp.loc[mask, 'effective_month'] = pd.to_datetime(temp.loc[mask, 'effective_month'].replace(
    'beginning', '2022-01-01'), format='%b-%Y', errors='coerce')
# temp['effective_month'] = temp['effective_month'].dt.strftime('%b-%Y')